In [1]:
partition = 100

In [2]:
import sys
from train import main
from itertools import product  
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:

"""
n_tree_values = [5, 10, 15, 20]
tree_depth_values = [3, 5, 10]
batch_size_values = [256, 512, 1000]

best_score = 0
best_config = {}

for n_tree, tree_depth, batch_size in product(n_tree_values, tree_depth_values, batch_size_values):
    print(f"\nRunning: n_tree={n_tree}, tree_depth={tree_depth}, batch_size={batch_size}")
    sys.argv = [
        'train.py',
        '-dataset', f'gtd{partition}',
        '-n_class', '30',
        '-gpuid', '0',
        '-n_tree', str(n_tree),
        '-tree_depth', str(tree_depth),
        '-batch_size', str(batch_size),
        '-epochs', '200',
        '-jointly_training'
    ]
    
    _, _, _, _ = main()

    # Read best score from file (assumes one run per file)
    result_file = f"results/result_gtd{partition}"
    with open(result_file, "r") as f:
        lines = f.readlines()
        for line in lines:
            if "Best Accuracy" in line:
                acc = float(line.split()[2])
                if acc > best_score:
                    best_score = acc
                    best_config = {
                        'n_tree': n_tree,
                        'tree_depth': tree_depth,
                        'batch_size': batch_size
                    }
print("\nBest hyperparameter configuration:")
print(best_config)
print(f"Best accuracy: {best_score}")
"""

'\nn_tree_values = [5, 10, 15, 20]\ntree_depth_values = [3, 5, 10]\nbatch_size_values = [256, 512, 1000]\n\nbest_score = 0\nbest_config = {}\n\nfor n_tree, tree_depth, batch_size in product(n_tree_values, tree_depth_values, batch_size_values):\n    print(f"\nRunning: n_tree={n_tree}, tree_depth={tree_depth}, batch_size={batch_size}")\n    sys.argv = [\n        \'train.py\',\n        \'-dataset\', f\'gtd{partition}\',\n        \'-n_class\', \'30\',\n        \'-gpuid\', \'0\',\n        \'-n_tree\', str(n_tree),\n        \'-tree_depth\', str(tree_depth),\n        \'-batch_size\', str(batch_size),\n        \'-epochs\', \'200\',\n        \'-jointly_training\'\n    ]\n    \n    _, _, _, _ = main()\n\n    # Read best score from file (assumes one run per file)\n    result_file = f"results/result_gtd{partition}"\n    with open(result_file, "r") as f:\n        lines = f.readlines()\n        for line in lines:\n            if "Best Accuracy" in line:\n                acc = float(line.split()[2])\

In [4]:
#{'n_tree': 15, 'tree_depth': 10, 'batch_size': 256}

sys.argv = [
    'train.py',
    '-dataset', f'gtd{partition}',
    '-n_class', '30',
    '-gpuid', '0',
    '-n_tree', '15',
    '-tree_depth', '10',
    '-batch_size', '256',
    '-epochs', '200',
    '-jointly_training'
]

preds, targets, labels, epoch_logs = main()


Use gtd100 dataset


Training Epochs:   0%|          | 0/200 [00:00<?, ?it/s]

Train Epoch: 1 [0/2100 (0%)]	Loss: 3.401443


Training Epochs:   0%|          | 1/200 [00:00<02:14,  1.48it/s]


Test set: Average loss: 3.3996, Accuracy: 88/900 (0.0978)

Train Epoch: 2 [0/2100 (0%)]	Loss: 3.399621


...


Test set: Average loss: 1.9746, Accuracy: 491/900 (0.5456)

Train Epoch: 199 [0/2100 (0%)]	Loss: 1.148358


Training Epochs: 100%|█████████▉| 199/200 [01:13<00:00,  2.72it/s]


Test set: Average loss: 1.9704, Accuracy: 476/900 (0.5289)

Train Epoch: 200 [0/2100 (0%)]	Loss: 1.113663


Training Epochs: 100%|██████████| 200/200 [01:14<00:00,  2.69it/s]


Test set: Average loss: 1.9694, Accuracy: 474/900 (0.5267)



In [5]:
from sklearn.metrics import classification_report

print(classification_report(targets, preds))

              precision    recall  f1-score   support

           0       0.45      0.73      0.56        30
           1       0.62      0.87      0.72        30
           2       0.67      0.60      0.63        30
           3       0.41      0.60      0.49        30
           4       0.73      0.53      0.62        30
           5       0.80      0.80      0.80        30
           6       0.40      0.47      0.43        30
           7       0.34      0.37      0.35        30
           8       1.00      0.80      0.89        30
           9       1.00      0.70      0.82        30
          10       0.57      0.90      0.70        30
          11       0.54      0.67      0.60        30
          12       0.50      0.50      0.50        30
          13       0.80      0.67      0.73        30
          14       0.48      0.43      0.46        30
          15       0.11      0.17      0.13        30
          16       0.40      0.67      0.50        30
          17       1.00    

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [6]:
def plot_confusion_matrix(y_true, y_pred, labels, partition):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(labels)))
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

    plt.figure(figsize=(18, 16))
    sns.heatmap(cm_normalized,
                annot=True,
                fmt=".2f",
                xticklabels=labels,
                yticklabels=labels,
                cmap="viridis",
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8})

    plt.title(f"Normalized Confusion Matrix (Partition gtd{partition})", fontsize=18)
    plt.xlabel("Predicted Label", fontsize=14)
    plt.ylabel("True Label", fontsize=14)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()

    save_path = f"results/confusion_matrix_partition_gtd{partition}.png"
    plt.savefig(save_path, dpi=300)
    plt.close()

    print(f"Saved confusion matrix for partition gtd{partition} to {save_path}")



In [7]:
plot_confusion_matrix(targets, preds, labels, partition)

Saved confusion matrix for partition gtd100 to results/confusion_matrix_partition_gtd100.png
